In [31]:
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [26]:
# This normalized CSV is outputted from R 
# Model_Norm = pd.read_csv("Model_Norm.csv", )
# Model_Norm

In [27]:
########### CONNECT TO ThalesStockPredictor SQLServer DB

# Define connection string
connStr = 'mssql+pyodbc://@MSI/ThalesStockPredictor?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

# Create SQLAlchemy engine
engine = create_engine(connStr)

# Define the query
query = """
SELECT * FROM vw_COMBINED_MODEL
ORDER BY FK_DT_Date desc
"""

# Execute the query and assign the result to a pandas DataFrame
Model_Data = pd.read_sql(query, engine)

# Close the SQL Server Connection
engine.dispose()

In [28]:
# View the data
Model_Data

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,THA_NextDay_Close,SPI_Open,SPI_High,...,SPI_SMA,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV
0,2024-03-01,137.500000,138.250000,135.399994,135.399994,135.399994,236008.0,NaN,5098.509766,5121.680176,...,5022.724438,5132.551599,4912.897278,7780.287500,8047.620766,7512.954234,2.310308e+10,2.310308e+10,1.297798e+12,8.105322e+09
1,2024-02-29,137.250000,137.300003,135.699997,137.050003,137.050003,984695.0,135.399994,5085.359863,5104.990234,...,5011.961426,5123.232730,4900.690122,7763.191992,8032.810869,7493.573116,2.310308e+10,2.310308e+10,1.296782e+12,8.105322e+09
2,2024-02-28,137.000000,137.149994,135.850006,136.600006,136.600006,186668.0,137.050003,5067.200195,5077.370117,...,4999.430420,5126.106344,4872.754496,7746.257983,8014.985481,7477.530486,2.300034e+10,2.300034e+10,1.291562e+12,8.105322e+09
3,2024-02-27,138.899994,139.100006,136.000000,136.850006,136.850006,254221.0,136.600006,5074.600098,5080.689941,...,4992.190942,5118.492300,4865.889585,7731.375977,7984.059230,7478.692723,2.294370e+10,2.294370e+10,1.295351e+12,8.052565e+09
4,2024-02-26,135.500000,139.250000,135.050003,138.750000,138.750000,304159.0,136.850006,5093.000000,5097.660156,...,4984.678442,5107.263494,4862.093391,7717.829492,7949.717954,7485.941031,2.288397e+10,2.288397e+10,1.291425e+12,8.002167e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6209,2000-01-07,33.500000,35.119999,33.349998,35.049999,19.815241,506662.0,35.200001,1403.449951,1441.469971,...,1420.333984,1472.471683,1368.196286,5611.762012,5993.606240,5229.917784,-8.448290e+07,-8.448290e+07,2.394000e+09,0.000000e+00
6210,2000-01-06,33.000000,33.400002,32.599998,33.160000,18.746744,178098.0,35.049999,1402.109985,1411.900024,...,1415.049988,1468.714715,1361.385261,5629.800049,6060.764238,5198.835860,-1.405418e+08,-1.405418e+08,1.168800e+09,0.000000e+00
6211,2000-01-05,33.599998,34.599998,32.000000,32.000000,18.090944,429936.0,33.160000,1399.420044,1413.270020,...,1418.916667,1481.853345,1355.979989,5689.696777,6128.436294,5250.957260,-9.936120e+07,-9.936120e+07,7.650003e+07,0.000000e+00
6212,2000-01-04,35.950001,35.950001,33.099998,33.750000,19.080292,435639.0,32.000000,1455.219971,1455.219971,...,1427.320007,1506.233021,1348.406994,5794.695068,6141.672504,5447.717633,-4.667840e+07,-4.667840e+07,-1.009000e+09,0.000000e+00


In [33]:
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6214 entries, 0 to 6213
Data columns (total 86 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   FK_DT_Date         6214 non-null   object 
 1   THA_Open           6214 non-null   float64
 2   THA_High           6214 non-null   float64
 3   THA_Low            6214 non-null   float64
 4   THA_Close          6214 non-null   float64
 5   THA_Adj_Close      6214 non-null   float64
 6   THA_Volume         6214 non-null   float64
 7   THA_NextDay_Close  6213 non-null   float64
 8   SPI_Open           6035 non-null   float64
 9   SPI_High           6035 non-null   float64
 10  SPI_Low            6035 non-null   float64
 11  SPI_Close          6035 non-null   float64
 12  SPI_Adj_Close      6035 non-null   float64
 13  SPI_Volume         6035 non-null   float64
 14  FRA_Open           6178 non-null   float64
 15  FRA_High           6178 non-null   float64
 16  FRA_Low            6178 

In [40]:
# Copy original data frame for normalization
Model_Norm = Model_Data.copy()

In [ ]:
# Split Dataframe into Training, Validation, Testing


In [39]:
# Assuming df is your DataFrame and 'col1' and 'col2' are the columns you don't want to normalize
cols_to_normalize = [col for col in Model_Norm.columns if col not in ['FK_DT_Date', 'THA_NextDay_Close']]

scaler = MinMaxScaler()

Model_Norm[cols_to_normalize] = scaler.fit_transform(Model_Norm[cols_to_normalize])
